In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import time

In [2]:
baseURL = 'https://www.imdb.com'

## Get genre
url = "https://www.imdb.com/chart/top/"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find_all('li', class_='subnav_item_main')
listGenre = [{'Name':item.text.replace('\n','').strip(),'URL':baseURL+item.a['href']} for item in results]
df_genre = pd.DataFrame(listGenre)
print ("DF genres with url:")
df_genre

DF genres with url:


,Name,URL
0,Action,https://www.imdb.com/search/title?genres=actio...
1,Adventure,https://www.imdb.com/search/title?genres=adven...
2,Animation,https://www.imdb.com/search/title?genres=anima...
3,Biography,https://www.imdb.com/search/title?genres=biogr...
4,Comedy,https://www.imdb.com/search/title?genres=comed...
5,Crime,https://www.imdb.com/search/title?genres=crime...
6,Drama,https://www.imdb.com/search/title?genres=drama...
7,Family,https://www.imdb.com/search/title?genres=famil...
8,Fantasy,https://www.imdb.com/search/title?genres=fanta...
9,Film-Noir,https://www.imdb.com/search/title?genres=film_...


In [3]:
## ---------------------------------------------
def GetDataPerGenre(genre, uri):
    time.sleep(0.2)
    page = requests.get(uri)
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.find_all('div', class_='lister-item mode-advanced')
    data = []
    for el in content:
        name = el('h3',class_='lister-item-header')[0].a.text ## get item without list
        i = el('h3',class_='lister-item-header')[0].a['href'].split('/')[2]  ## get item without list
        year = el('h3',class_='lister-item-header')[0].find('span', class_='lister-item-year').text.replace('(','').replace(')','')
        imdb_rating = el('div',class_='ratings-bar')[0].find('div', class_='ratings-imdb-rating').strong.text
        director = el('div', class_='lister-item-content')[0].find('p', class_='').find_all('a')[0].text
        #stars = el('div', class_='lister-item-content')[0].find('p', class_='').find_all('a')[1].text + ',' + 
        votes = el('div', class_='lister-item-content')[0].find('p',class_='sort-num_votes-visible').find_all('span',{"name":"nv"})[0].text
        malesRating, malesCount, femalesRating, femalesCount = GetRatingDataByID(i)
        try:
            gross = el('div', class_='lister-item-content')[0].find('p',class_='sort-num_votes-visible').find_all('span',{"name":"nv"})[1].text
        except:
            gross = np.nan
            
        data.append({'Name':name,'Genre':genre,'ID':i,'Year':year,'Imdb Rating':imdb_rating,'Director':director, 
                     'Votes':votes,'Gross':gross, 'malesRating':malesRating, 'malesCount':malesCount, 
                     'femalesRating':femalesRating ,'femalesCount':femalesCount})
    return data
## ---------------------------------------------
def GetRatingDataByID(id):
    time.sleep(1)
    uri = "https://www.imdb.com/title/{0}/ratings/?ref_=tt_ov_rt".format(id)
    page = requests.get(uri)
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.find_all('table')[1]
    
    males = content.find_all('tr')[2]
    malesRating = males.find('div', class_='bigcell').text
    malesCount = males.find('div', class_='smallcell').text.strip()

    females = content.find_all('tr')[3]
    femalesRating = females.find('div', class_='bigcell').text
    femalesCount = females.find('div', class_='smallcell').text.strip()

    return (malesRating, malesCount, femalesRating, femalesCount)
## ---------------------------------------------
data = []
for item in listGenre:
    data.extend(GetDataPerGenre(item['Name'], item['URL']))

print (len(data))
df = pd.DataFrame(data)
df

1031


,Name,Genre,ID,Year,Imdb Rating,Director,Votes,Gross,malesRating,malesCount,femalesRating,femalesCount
0,The Dark Knight,Action,tt0468569,2008,9.0,Christopher Nolan,"2,556,384",$534.86M,9.0,"1,476,807",8.7,"264,598"
1,The Lord of the Rings: The Return of the King,Action,tt0167260,2003,9.0,Peter Jackson,"1,777,166",$377.85M,9.0,"1,027,904",8.9,"218,150"
2,K.G.F: Chapter 2,Action,tt10698680,2022,8.9,Prashanth Neel,"93,320",NaN,8.3,"13,286",7.5,428
3,Everything Everywhere All at Once,Action,tt6710474,2022,8.8,Dan Kwan,"39,711",NaN,8.7,"17,826",8.7,"3,781"
4,Inception,Action,tt1375666,2010,8.8,Christopher Nolan,"2,269,287",$292.58M,8.8,"1,260,681",8.7,"293,467"
...,...,...,...,...,...,...,...,...,...,...,...,...
1026,True Grit,Western,tt0065126,1969,7.4,Henry Hathaway,"46,926",$31.13M,7.4,"32,771",7.5,"4,138"
1027,The Ballad of Buster Scruggs,Western,tt6412452,2018,7.3,Ethan Coen,"142,189",NaN,7.3,"86,392",7.2,"10,974"
1028,Bacurau,Western,tt2762506,2019,7.3,Juliano Dornelles,"26,525",NaN,7.3,"16,456",7.6,"2,542"
1029,The Proposition,Western,tt0421238,2005,7.3,John Hillcoat,"52,397",$1.90M,7.3,"39,814",7.3,"3,532"


In [4]:
# Export data to a csv file
df.to_csv("IMDb_rating_data.csv")